# Neural Network Car Race Game
### Author: Huanjia Liu

## Introduction
My project for CS 345 is a neural Network Car Race Game. It's demo built by Unity3d, using neural network and genetic algorithm to help the car learn how to play the game by itself. So there are three main missions during my project period. 

1. **Building a steerable race game demo in Unity3d**

2. **Creating neural network class in C#**

At first, I considered about using C# to call Python script, so I can build a neural network by using Python library easily. However, some unexpected situations occurred during the exploration. <https://docs.unity3d.com/Packages/com.unity.scripting.python@2.0/manual/inProcessAPI.html>

![jupyter](limination.png)

Thus, I had to write a nerual network by myself using a plenty of *for loop* in C#

3. **Designing a genetic algorithm**

Next, I will describe these three processes in detail separately

## Unity 3D Part
Unity 3d is a cross-platform game engine developed by Unity Technologies. Here I use it to build my race game scene, including a race track using original terrain, the car demo(a cube), and the race track routine. 

### Car

It needs to have two systems as a self-drive race car: a power system and a monitoring system. The power system can be achieved by Unity3d API. ([Transform.Translate](https://docs.unity3d.com/ScriptReference/Transform.Translate.html) [Transform.Rotate](https://docs.unity3d.com/ScriptReference/Transform.Rotate.html) ) And monitoring system is implemented by the class *raycast*, like the figure below.

![sensor] (sensor.png)

It will shoot a ray at a certain angle from the designated position, here the position is the center point of the car, and there are five sensor direction: (0°, 45°, 45°, -90°, 90°). I also set the maximum value of the ray to be infinite, and added the mask layer to avoid the influence of objects other than terrain, since we will create 100 cars each time, sensors need to ignore all of them.

The code of power and monitoring system are in *Race_Car_center.cs*

### Race track routine

The race track routine is the line with some spheres in the figure below, powered by [**ToolBuddy**](https://forum.curvyeditor.com/) 
 which can help to return the distance progress the car has finished.  So it won't affect the state of the car when running.

 The code are running in C# script *nearest.cs*
 
 ![track] (track.png)  


## Neural Network Part

### Neural class

A neural network is a series of algorithms that endeavors to recognize underlying relationships in a set of data through a process that mimics the way the human brain operates. Neural is the essential component in a neural network, which is also the starting point of my code. Here I create a neural class, including the three variables: weight array, bias, and value. 
 
 ![neural] (nerual.png)  

The weight array store the weight data between this neural to the previous layer. For example, here is a neural network with the structure 3x3x2. The weight array in the neural pointed by the blue arrow store the weight 1,2,3.

 ![neural] (neural1.png)  

The value stores the value after calculation, which is equal 
 $$Value_{current} =  activation((\sum_{n=0}^{length} W_n * Value_n) + Bias)$$
Value_n is the value in previous layer. Do not forget that the activation function needs to be applied. Here I use the sigmoid function. The code is shown below.

![sigmoid] (sigmoid.png)  

### Neural Network

Then we can combine them together to form a neural network. In my project, there are five inputs and two outputs. Five inputs are the distance provided by Raycast; two outputs are used to control the speed and the direction separately. We don't need to worry about which output need to do which job, the neural network will assign by itself. 

As for the hidden layer, from the book "Data Mining Techniques: For Marketing, Sales, and Customer Relationship Management", the number of hidden units should not exceed twice the number of units in the input layer, so I only set 1 hidden layer with 3 neural (1/2 of input unit), but it seems work quite well.

There are two functions in my neural network class. They are the *initial()* responsible for initialization and the *float[] forward(float[] input)* responsible for forwarding propagation. 

During the initialization, we assign a random number between -0.5 to 0.5  to weight and bias. Be careful. The initialization of the array needs to specify the length in C#, so we need to determine the structure of our neural network first.

![ini] (ini.png) 

The forward propagation is quite easier. Just follow the equation I mentioned above. Here are the code.

![fow] (forwar.png) 

All the code above can be accessed in *NeuralNetwork.cs*

## Genetic Algorithm

The final step is the design of the genetic algorithm. From the article ["Optimization of the Mutation on Genetic Algorithms"](https://medium.com/@louisn_23157/optimization-mutation-genetic-algorithm-40247f8ccb8) written by Louis Nicolle, we can learn that selection and reproduction are the most critical procedures. The purpose of selection is to select individuals with excellent performance, while reproduction is carried out on the basis of excellent individuals, aiming to mutate better individuals.

In my project, the "selection" step is dependent on the grade, and the grade is generated by the routine I have mentioned above, which will calculate the distance progress of the car in real-time.  We choose the highest car as our maternal individual, then apply [uniform mutation](https://en.wikipedia.org/wiki/Mutation_(genetic_algorithm)). Of course, the variation cannot appear in all "genes." Otherwise, it would be meaningless for us to choose the best individual. The mutation rate should also be applied. In Louis's article, he uses a random number to express probability like the code below. When an individual mutates, the value of one of its genes is changed.

In [ ]:
if (100 * random.random() < mutationRate):
    mutate()

I borrowed his method and set the probability to 8% with random numbers. If the mutation happened, a random number between -0.5 to 0.5 be added. Initialization is only applied at the first generation, but the mutation will be applied to all the rest interactions. Here is my code.

![mutate](mutate.png)

The detail code can be found in *center_test.cs*

Now we can talk about our logic sequence. 
When the game starts, we initialize and generate the first generation of 100 cars. The parameters of the car's neural network are randomly distributed between -0.5 and 0.5, and the selection process begins. When the car hits a wall or runs for more than 15 seconds, the score is settled and submitted. The highest score is selected by the control center. If the person with the highest score has not completed 100% of the distance (didn't reach the endpoint), then the mutation is performed. Although the new 100 mutated cars have different parameters, the overall parameters are similar to those of the best performers in the previous round. Go through the selection process again. Continue to loop until there is a car that does not hit the wall to reach the end within 15 seconds.

![flow](flow.png)